In [ ]:
workflow = {
    1: "data (prepare and load)",
    2: "build model",
    3: "fitting the model to data (training)",
    4: "making predictions and evaluating a model (inference)",
    5: "saving and loading a model",
    6: "putting it all together"
}
workflow

In [ ]:
import torch
from torch import nn # nn contain all of PT building blocks
import matplotlib.pyplot as plt

# Py version
torch.__version__

In [ ]:
torch.version.cuda

In [ ]:
torch.cuda.is_available()

## 1. Data (preparing and loading)

1. Get data into numerical representation
2. Build a model to learn patterns in the numerical representation

Create known data using linear regression formula  
linear regression formula creates a straight line with known parameters

In [ ]:
# Create *known* parameters
weight = 0.7
bias = 0.3 

# create range
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

In [ ]:
len(X), len(y)

### Splitting data into training and test sets

In [ ]:
# create train/test split
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

### Visualize data

In [ ]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
    """
    Plots training data, test data and compares predictions.
    """
    plt.figure(figsize=(10,7))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

    # Plot test data in green
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

    # if predictions
    if predictions is not None:
        # Plot the predictions if they exist
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")
    
    plt.legend(prop={"size": 14});

In [ ]:
plot_predictions();

## 2. Build model

model explanation:
* starts with random values (weight & bias)
* look at training data and adjust the random values to better represent the ideal values
* accomplished through 2 algos:
    1. Gradient descent
    2. Backpropagation

In [ ]:
# linear regression model
# most things in PyTorch inherit from nn.Module
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, # start with random value
                                                requires_grad=True, # update via gradient descent
                                                dtype=torch.float)) 
        self.bias == nn.Parameter(torch.randn(1,
                                              requires_grad=True,
                                              dtype=torch.float))
        
        # forward method to define the computation in model
        def forward(self, x: torch.Tensor) -> torch.Tensor:
            return self.weights * x + self.bias # linear regression formula 
        

### PyTorch model building essentials

* torch.nn - contains building blocks for computational graphs
* torch.nn.Parameter - what parameters should the model try to learn, often a PT layer from torch.nn will set these
* torch.nn.Module - base clas for nn modules, if subclassed always override the foward method
* torch.optim - is where to find the optimizers for gradient descent
* def forward() - all nn.Module subclasses require this to be overriden, defines what happens in the forward computation